**Load Data**

This project uses two excel files and States shapefile

The first dataset contains US Highway fatalities 2019-2021

The second dataset contains Per Capital alcohol consumption for 43 states 2019-2021. The data for 7 states was missing or incomplete and hence omitted.

The third dataset contains States geographic data from mapping

In [1]:
#import the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Viridis256
import geopandas as gpd
import zipfile
import os

# Set the style for Matplotlib and Seaborn
sns.set(style="whitegrid")

In [2]:
#Read CSV file 1 containing US Highway Fatalities 2019-2021
US_Fatalities= pd.read_excel("datasets/Highway_Deaths.xlsx")
#display the first 5 rows of the US Fatalities dataset
US_Fatalities.head()

,State,Year_2019,Year_2020,Year_2021
0,Alabama,1251,1215,1312
1,Alaska,90,85,91
2,Arizona,1284,1393,1662
3,Arkansas,668,856,927
4,California,4855,5374,5897


In [3]:
#check the US_Fatalities df info
US_Fatalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   State      51 non-null     object
 1   Year_2019  51 non-null     int64 
 2   Year_2020  51 non-null     int64 
 3   Year_2021  51 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.7+ KB


In [4]:
#Read the per capital alcohol concumption per state between 2019 and 2021
Alcohol_consumption= pd.read_excel("datasets/Per_Capita_Alcohol_Consumption.xlsx")
#display the first 5 rows of the dataset
Alcohol_consumption.head()

,State_District,2019_c,2020_c,2021_c
0,Alabama,2.68,2.50,2.65
1,Alaska,3.24,3.42,3.65
2,Arizona,2.42,2.34,2.52
3,Arkansas,2.33,2.21,2.36
4,California,2.37,2.33,2.48


In [5]:
# Check the dataset's information
Alcohol_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State_District  43 non-null     object 
 1   2019_c          43 non-null     float64
 2   2020_c          43 non-null     float64
 3   2021_c          43 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.5+ KB


**Clean, Normalize and merge the datasets**

Since the alcohol consumption by state was already normalized to account for population differences, it was crucial to normalize the US fatalities dataset and eliminate those States whose Alcohol consumption data was not available.

In [6]:

# Extract the columns to be normalized
columns_to_normalize = ['Year_2019', 'Year_2020', 'Year_2021']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
US_Fatalities[columns_to_normalize] = scaler.fit_transform(US_Fatalities[columns_to_normalize])

#Display the first 5 rows of the normalized dataset

US_Fatalities.head()

,State,Year_2019,Year_2020,Year_2021
0,Alabama,0.237216,0.208691,0.190491
1,Alaska,0.011472,0.006617,0.005603
2,Arizona,0.243632,0.240522,0.243489
3,Arkansas,0.123858,0.144492,0.132193
4,California,0.937974,0.952432,0.884767


In [7]:
# Extract states from US_Fatalities and Alcohol_consumption datasets
fatalities_states = US_Fatalities['State'].tolist()
alcohol_states = Alcohol_consumption['State_District'].tolist()

# Find missing states by comparing columns
missing_states = set(fatalities_states) ^ set(alcohol_states)

#Print out the missing states

print("Missing States:", missing_states)


Missing States: {'Utah', 'Texas', 'West Virginia', 'Vermont', 'Washington', 'Wisconsin', 'Virginia', 'Wyoming'}


In [8]:
# Find common states between US_Fatalities and Alcohol_consumption datasets
common_states = set(fatalities_states) & set(alcohol_states)

# Filter US_Fatalities dataset
fatalities_data_cleaned = US_Fatalities[US_Fatalities['State'].isin(common_states)]

#Display the cleaned dataset
fatalities_data_cleaned.head()

,State,Year_2019,Year_2020,Year_2021
0,Alabama,0.237216,0.208691,0.190491
1,Alaska,0.011472,0.006617,0.005603
2,Arizona,0.243632,0.240522,0.243489
3,Arkansas,0.123858,0.144492,0.132193
4,California,0.937974,0.952432,0.884767


In [9]:
#Confirm that the dataset now contains 43 rows
fatalities_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43 entries, 0 to 42
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   State      43 non-null     object 
 1   Year_2019  43 non-null     float64
 2   Year_2020  43 non-null     float64
 3   Year_2021  43 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.7+ KB


**Merge the two dataset oon State column**

The two datasets share the column State/State_District, making it easy to merge them

In [10]:
# Merge datasets on the 'State' column
merged_data = pd.merge(fatalities_data_cleaned, Alcohol_consumption, left_on='State', right_on='State_District', how='inner')

# Drop the duplicate 'State_District' column, if needed
merged_data = merged_data.drop('State_District', axis=1)

# Display the merged dataset
merged_data.head()


,State,Year_2019,Year_2020,Year_2021,2019_c,2020_c,2021_c
0,Alabama,0.237216,0.208691,0.190491,2.68,2.50,2.65
1,Alaska,0.011472,0.006617,0.005603,3.24,3.42,3.65
2,Arizona,0.243632,0.240522,0.243489,2.42,2.34,2.52
3,Arkansas,0.123858,0.144492,0.132193,2.33,2.21,2.36
4,California,0.937974,0.952432,0.884767,2.37,2.33,2.48


In [11]:
#export the merged data to your Data folder
filepath="datasets/merged_data.csv"
merged_data.to_csv(filepath, index = False)

In [12]:
# Unzip the shapefile
zip_path = 'datasets/States_shapefile-shp.zip'
shapefile_path = 'datasets/States_shapefile-shp/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(shapefile_path)

# Get the path to the extracted shapefile
extracted_files = os.listdir(shapefile_path)
shapefile_name = [file for file in extracted_files if file.endswith('.shp')][0]
full_shapefile_path = os.path.join(shapefile_path, shapefile_name)

# Load the shapefile with U.S. states geometries
us_states = gpd.read_file(full_shapefile_path)
# Convert the 'State_Name' column to lowercase and remove leading/trailing spaces in the us_states GeoDataFrame
us_states['State_Name'] = us_states['State_Name'].str.lower().str.strip()

# Convert the 'State' column to lowercase and remove leading/trailing spaces in the merged_data DataFrame
merged_data['State'] = merged_data['State'].str.lower().str.strip()

# Merge the state geometries with the merged_data DataFrame containing average alcohol consumption
merged_map_data = us_states.merge(merged_data, left_on='State_Name', right_on='State', how='inner')

#Save the merged dataset to your Data folder
merged_map_data.to_csv("datasets/merged_data_map.csv", index =False)

**Cleaned Datasets**
- The Highway traffic fatalities dataset and Per Capital Alcohol consumption by State have been merged into one dataset
- The merged dataset is merged with the States' geographical data for choropleth map in visualization